# typing 모듈

- 파이썬은 동적 타입 언어로 변수의 타입을 선언하지 않아도 된다. 그러나 이런 특징은 규모가 큰 프로그램을 구현할 때 많은 문제를 야기시킨다.
- 그래서 파이썬은 타입 힌트를 제공하며 이를 이용해 코드의 가독성과 유지보수성을 높일 수 있다.
- **typing 모듈**은 타입 힌트를 언어 차원에서 지원하기 위해 파이썬 3.5 버전부터 추가된 표준 모듈이다. 
    - 타입 힌트를 설정 하기 위한 다양한 타입 클래스와 표현 방법을 제공한다.

## type hint 표현 방법

- 변수 선언: `variable이름: type [= value]`
- 함수 선언: `def function(parameter: type) -> return_type:`

## 주요 타입 클래스

**컬렉션 타입**
- `List[T]`: 리스트 타입을 표현하며 T는 요소의 타입이다.
- `Dict[K, V]`: 키 타입 K와 값 타입 V를 가지는 딕셔너리이다.
- `Tuple[T1, T2]`: 각 위치별 타입을 지정할 수 있는 튜플이다.
- `Set[T]`: 요소 타입이 T인 집합이다.

**특수 타입**
- `Optional[T]`: T 타입이거나 None일 수 있음을 나타낸다.
- `Union[T1, T2]`: T1 또는 T2 타입이 될 수 있음을 나타낸다.
- `Final[T]`: 재할당이 불가능한 상수 타입을 나타낸다.
- `Callable[[ArgType], ReturnType]`: 함수의 타입을 지정할 때 사용한다.
- `Iterable[T]`: T 타입의 요소를 반복할 수 있는 Iterable 객체를 나타낸다.

## Python 3.9+ 변경 사항

- Python 3.9 버전부터는 컬렉션 타입을 python 내장 타입을 직접 사용할 수 있게 되었다:
```python
numbers: list[int] = [1, 2, 3]
settings: dict[str, str] = {"name": "value"}
```

## Python 3.10+ 변경 사항
- `|` 연산자를 사용하여 Union 타입을 표현할 수 있다:
```python
# Python 3.10+ 이전
num: Union[int, float] = 1
# Python 3.10+ 이후
num: int | float = 1
```
- TypeAlias를 사용해 복잡한 타입을 단순화 할 수있다.
```python
from typing import TypeAlias
vector: TypeAlias = list[float]

num_list: vector
```

## 장점

- 코드의 가독성이 향상된다.
- IDE의 자동 완성 기능이 개선된다.
- 정적 타입 검사를 통해 잠재적 오류를 사전에 발견할 수 있다.


# Pydantic
- Pydantic은 Python 애플리케이션에서 데이터 유효성 검사를 쉽게 할 수 있도록 돕는 라이브러리이다. Langchain이나 FastAPI 등 다양한 Framework에서 데이터 유효성검사나 스키마 정의를 위해 사용된다.
- 설치
  - `pip install pydantic`
- [공식문서](https://docs.pydantic.dev/latest/)
## 주요기능
- 타입 유효성 검사: 모델에 정의된 데이터 타입에 따라 자동으로 유효성을 검사한다.
- 자동 형 변환: 전달된 데이터가 정의된 타입과 일치하지 않을 경우 가능한 범위 내에서 자동으로 형 변환을 시도한다.
- 필드의 기본값 지원: 필드에 기본값을 설정하여 사용자가 데이터를 제공하지 않더라도 유효한 모델을 만들 수 있다.
- 복잡한 데이터 구조 지원: 리스트, 딕셔너리, 중첩된 모델과 같은 복잡한 데이터 구조를 지원한다.
- 데이터 직렬화 및 역직렬화: 모델 인스턴스를 JSON으로 직렬화하거나 JSON으로부터 역직렬화할 수 있다.

## 구현

- `BaseModel` 클래스를 상속받아 모델을 정의한다.
- 필드에 각 변수가 저장할 값의 타입을 지정한다.
- `Field`를 이용해 필요한 경우 필드에 유효성 검사를 위한 제약 조건을 추가한다.

In [ ]:
# 일반적인 방식으로 DataClass를 정의
class Person:

    def __init__(self, name, age, address): #속성들
        self.name = name
        if isinstance(age, int):
            self.age = age
        else:
            raise Exception("나이 타입이 아닙니다.")
        self.address = address

# p  = Person("홍길동", 20, "서울")
p  = Person("홍길동", "서른살", "서울")
print(p.name, p.age, p.address)

Exception: 나이 타입이 아닙니다.

In [ ]:
from pydantic import BaseModel
# Pydantic으로 Data Class 구현
class Member(BaseModel):
    # class 변수로 attribute들을 설정: Field
    name: str
    age: int | float    # Union -> int 또는 float 타입을 할당.
    nickname: str | None = None #  Optional. str 타입을 할당하는데 None을 가질 수도 있다.
    hobby: list[str]

In [6]:
m = Member(name="이순신", age=30, nickname="장군", hobby=["게임", "영화"])
print(m)
m.name, m.age

name='이순신' age=30 nickname='장군' hobby=['게임', '영화']


('이순신', 30)

In [9]:
m.age = 30.5
m.age

30.5

In [13]:
# m.age = "서른 다섯살"

In [14]:
m2 = Member(name="이순신", age="서른살", nickname="장군", hobby=["게임", "영화"])

ValidationError: 2 validation errors for Member
age.int
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='서른살', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
age.float
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='서른살', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing

## `Field` 함수를 이용해 추가 정보 및 제약 조건 추가.
- Pydantic의 `Field` 함수는 모델에 설명(Meta data)를 설정하고 유효성 검사 통과 조건을 설정하는 데 사용된다.
- **주요 파라미터**
  - 첫번째 인수로 `...`(Ellipsis) 가 들어가면 **필수 항목** 이다. 
  - **default**: 필드의 기본값을 지정. 기본값이 없을 경우 필드는 필수 항목으로 간주된다.
  - **default_factory**: default값을 생성하는 함수를 정의한다.  `default`와 동시에 사용할 수 없다.
  - **title**: 필드의 title을 지정한다. API 문서화나 데이터 스키마 생성 시 사용자가 필드의 의미를 쉽게 이해할 수 있도록 돕는 역할을 한다.
  - **description**: 필드에 대한 설명. 주로 API 문서화나 데이터 스키마 생성 시 사용자가 필드의 의미를 쉽게 이해할 수 있도록 돕는 역할을 한다.
  - **gt**: 조건 설정. 필드 값이 지정된 값보다 커야 한다. (greater than)
  - **ge**: 조건 설정. 필드 값이 지정된 값보다 크거나 같아야 한다. (greater than or equal)
  - **lt**: 조건 설정. 필드 값이 지정된 값보다 작아야 한다. (less than)
  - **le**: 조건 설정. 필드 값이 지정된 값보다 작거나 같아야 한다. (less than or equal)
  - **multiple_of**: 조건지정 필드 값이 지정된 값의 배수여야 한다.
  - **min_length**: 문자열 필드의 최소 길이를 지정.
  - **max_length**: 문자열 필드의 최대 길이를 지정.
  - **pattern**: 문자열 필드가 일치해야 하는 정규 표현식을 지정.

In [15]:
from pydantic import Field, BaseModel

class Member(BaseModel):
    # class 변수로 attribute들을 설정: Field
    name: str = Field(title="회원이름", description="회원의 이름을 저장")
    age: int | float = Field(title="회원나이", description="회원의 나이를 저장.", gt=0)
    nickname: str | None = Field(default=None, title="별명", description="회원의 별명")
    hobby: list[str] = Field(title="취미", description="회원의 취미", max_length=2, min_length=1)
    # max_length: 문자열-최대 글자수, 자료구조: 최대 원소개수 
    # default 가 없는 Field는 None을 가질 수없다. (필수 속성.)

In [16]:
m = Member(name="홍길동", age=20, nickname="의적", hobby=['독서', '게임'])
m

Member(name='홍길동', age=20, nickname='의적', hobby=['독서', '게임'])

In [ ]:
# m = Member( age=20, nickname="의적", hobby=['독서', '게임'])
# m = Member(name="홍길동", age=-20, nickname="의적", hobby=['독서', '게임'])
m = Member(name="홍길동", age=20, nickname="의적", hobby=['독서', '게임', "영화"])

ValidationError: 1 validation error for Member
hobby
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/too_short